# **Machine Learning II: Deep learning classifiers for urban sound data**
<img src="images/portrait.png" alt="Portrait" style="width:100%; height:auto;">

===================================================================================================================


## **Introduction**
Urban sound recognition is a crucial component in the development of intelligent systems for cities, supporting applications in public safety, environmental monitoring, and transportation. This project focuses on designing and evaluating deep learning classifiers for recognizing urban sounds using the UrbanSound8K dataset. This dataset contains audio samples across 10 classes commonly found in urban environments, such as "siren," "dog bark," and "street music." To address the classification challenge, we implement two neural network architectures: a **Recurrent Neural Network (RNN)**, which leverages temporal patterns in audio data, and a **Multilayer Perceptron (MLP)**, which classifies extracted features based on dense, fully connected layers. By comparing these models, we aim to identify which architecture is more effective in urban sound classification tasks, balancing accuracy, computational efficiency, and model complexity.


### **Abstract**

This project presents a deep learning approach for urban sound classification, utilizing the UrbanSound8K dataset to train and evaluate two neural network models: a Recurrent Neural Network (RNN) and a Multilayer Perceptron (MLP). Our data preparation pipeline applies comprehensive preprocessing steps, including signal normalization and feature extraction, to transform raw audio signals into structured representations suitable for deep learning input. Model architectures are iteratively optimized through careful adjustment of layer configurations, activation functions, and hyperparameters to enhance performance. 
Training strategies incorporate fine-tuning of optimizers, regularization techniques, and a robust 10-fold cross-validation approach to ensure reliable generalization and mitigate overfitting. Model performance is evaluated using classification accuracy and a cumulative confusion matrix across folds, providing insight into each model’s ability to handle distinct sound classes.

Training strategies included fine-tuning of optimizers and regularization techniques, alongside a robust **10-fold cross-validation** to assess generalization. Classification performance is evaluated based on accuracy and the confusion matrix across folds, providing insights into each model's strengths in handling distinct sound classes. 

### **Objective of the work**
The primary objective of this project is to develop and assess the efficacy of two distinct neural network architectures—RNN and MLP—for urban sound classification. By conducting a comparative analysis, we aim to provide understanding of each model's performance, identifying the trade-offs in accuracy, computational demands, and architectural complexity. 

### **Structure of the work**
The project is structured into the following sections:
1. **Data Preparation**: This section outlines the steps taken to preprocess the UrbanSound8K dataset
2. **Model Architectures**: A detailed description of the RNN and MLP models...
    - **RNN Architecture**: ...
    - **MLP Architecture**: ...
3. **Training Strategies**: This section discusses the training approaches used for both models...
4. **Evaluation Metrics**: A description of the metrics used to evaluate model performance...
5. **Results and Discussion**: A presentation of the results obtained from the models, including accuracy and ...

## **Data Preparation**

In [1]:
############## LIBRARIES ###############
import soundata
import librosa
import numpy as np
import pandas as pd

In [ ]:

dataset = soundata.initialize('urbansound8k', data_home=r'C:\Users\34660\Desktop\DeepLearningAudios')
dataset.download()  # Dataset is downloaded to my directory
dataset.evaluate()

INFO: Downloading ['all', 'index']. Index is being stored in C:\Users\34660\OneDrive - Universidad de Oviedo\Ciencia e Ingeniería de Datos\3º\Deep Learning\Project\MCII-Project\.venv\Lib\site-packages\soundata\datasets\indexes, and the rest of files in C:\Users\34660\Desktop\DeepLearningAudios
INFO: [all] downloading UrbanSound8K.tar.gz


  1%|          | 43.1M/5.61G [00:18<57:04, 1.75MB/s]   

In [6]:
dataset.validate()

FileNotFoundError: This dataset's index must be downloaded. Did you run .download()?